# A Gentle Introduction to RAG Applications

This notebook creates a simple RAG (Retrieval-Augmented Generation) system to answer questions from a PDF document using an open-source model.

In [12]:
PDF_FILE = "paul.pdf"

# We'll be using Llama 3.1 8B for this example.
MODEL = "llama3.1"

## Loading the PDF document

Let's start by loading the PDF document and breaking it down into separate pages.

<img src='images/documents.png' width="1000">

In [13]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_FILE)
pages = loader.load()

print(f"Number of pages: {len(pages)}")
print(f"Length of a page: {len(pages[1].page_content)}")
print("Content of a page:", pages[1].page_content)

Number of pages: 9
Length of a page: 3217
Content of a page: 10% a week. And while 110 may not seem much better than 100,if you keep growing at 10% a week you'll be surprised how bigthe numbers get. After a year you'll have 14,000 users, and after2 years you'll have 2 million.You'll be doing different things when you're acquiring users athousand at a time, and growth has to slow down eventually. Butif the market exists you can usually start by recruiting usersmanually and then gradually switch to less manual methods. [3]Airbnb is a classic example of this technique. Marketplaces are sohard to get rolling that you should expect to take heroic measuresat first. In Airbnb's case, these consisted of going door to door inNew York, recruiting new users and helping existing ones improvetheir listings. When I remember the Airbnbs during YC, I picturethem with rolly bags, because when they showed up for tuesdaydinners they'd always just flown back from somewhere.FragileAirbnb now seems like an 

## Splitting the pages in chunks

Pages are too long, so let's split pages into different chunks.

<img src='images/splitter.png' width="1000">


In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)

chunks = splitter.split_documents(pages)
print(f"Number of chunks: {len(chunks)}")
print(f"Length of a chunk: {len(chunks[1].page_content)}")
print("Content of a chunk:", chunks[1].page_content)


Number of chunks: 32
Length of a chunk: 1494
Content of a chunk: July 2013One of the most common types of advice we give at Y Combinatoris to do things that don't scale. A lot of would-be founders believethat startups either take off or don't. You build something, makeit available, and if you've made a better mousetrap, people beat apath to your door as promised. Or they don't, in which case themarket must not exist. [1]Actually startups take off because the founders make them takeoff. There may be a handful that just grew by themselves, butusually it takes some sort of push to get them going. A goodmetaphor would be the cranks that car engines had before theygot electric starters. Once the engine was going, it would keepgoing, but there was a separate and laborious process to get itgoing.RecruitThe most common unscalable thing founders have to do at thestart is to recruit users manually. Nearly all startups have to. Youcan't wait for users to come to you. You have to go out and getthe

## Storing the chunks in a vector store

We can now generate embeddings for every chunk and store them in a vector store.

<img src='images/vectorstore.png' width="1000">


In [15]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=MODEL)
vectorstore = FAISS.from_documents(chunks, embeddings)

## Setting up a retriever

We can use a retriever to find chunks in the vector store that are similar to a supplied question.

<img src='images/retriever.png' width="1000">



In [16]:
retriever = vectorstore.as_retriever()
retriever.invoke("What can you get away with when you only have a small number of users?")

[Document(metadata={'source': 'paul.pdf', 'page': 7}, page_content="started are not merely a necessary evil, but change the companypermanently for the better. If you have to be aggressive aboutuser acquisition when you're small, you'll probably still beaggressive when you're big. If you have to manufacture your ownhardware, or use your software on users's behalf, you'll learnthings you couldn't have learned otherwise. And mostimportantly, if you have to work hard to delight users when youonly have a handful of them, you'll keep doing it when you have alot."),
 Document(metadata={'source': 'paul.pdf', 'page': 0}, page_content='Want to start a startup? Get funded by Y Combinator.'),
 Document(metadata={'source': 'paul.pdf', 'page': 7}, page_content='Notes[1] Actually Emerson never mentioned mousetraps specifically. Hewrote "If a man has good corn or wood, or boards, or pigs, tosell, or can make better chairs or knives, crucibles or churchorgans, than anybody else, you will find a broad h

## Configuring the model

We'll be using Ollama to load the local model in memory. After creating the model, we can invoke it with a question to get the response back.

<img src='images/model.png' width="1000">

In [17]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=MODEL, temperature=0)
model.invoke("Who is the president of the United States?")

AIMessage(content='As of my last update in April 2023, Joe Biden is the President of the United States. He took office on January 20, 2021, succeeding Donald Trump as the 46th President of the United States. Please note that this information might change over time due to elections or other political developments.', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-11T15:56:24.740654949Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1628813229, 'load_duration': 22064348, 'prompt_eval_count': 20, 'prompt_eval_duration': 40750000, 'eval_count': 65, 'eval_duration': 1520747000}, id='run-c453f60f-b4c3-4ed8-84db-c715f94eb563-0', usage_metadata={'input_tokens': 20, 'output_tokens': 65, 'total_tokens': 85})

## Parsing the model's response

The response from the model is an `AIMessage` instance containing the answer. We can extract the text answer by using the appropriate output parser. We can connect the model and the parser using a chain.

<img src='images/parser.png' width="1000">


In [18]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
print(chain.invoke("Who is the president of the United States?"))

As of my last update in April 2023, Joe Biden is the President of the United States. He took office on January 20, 2021, succeeding Donald Trump as the 46th President of the United States. Please note that this information might change over time due to elections or other political developments.


## Setting up a prompt

In addition to the question we want to ask, we also want to provide the model with the context from the PDF file. We can use a prompt template to define and reuse the prompt we'll use with the model.


<img src='images/prompt.png' width="1000">

In [19]:
from langchain.prompts import PromptTemplate

template = """
You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: Here is some context

Question: Here is a question



## Adding the prompt to the chain

We can now chain the prompt with the model and the parser.

<img src='images/chain1.png' width="1000">

In [20]:
chain = prompt | model | parser

chain.invoke({
    "context": "Anna's sister is Susan", 
    "question": "Who is Susan's sister?"
})


'Anna.'

## Adding the retriever to the chain

Finally, we can connect the retriever to the chain to get the context from the vector store.

<img src='images/chain2.png' width="1000">

In [22]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

## Using the chain to answer questions

Finally, we can use the chain to ask questions that will be answered using the PDF document.

In [23]:
questions = [
    "What can you get away with when you only have a small number of users?",
    "What's the most common unscalable thing founders have to do at the start?",
    "What's one of the biggest things inexperienced founders and investors get wrong about startups?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print("*************************\n")

Question: What can you get away with when you only have a small number of users?
Answer: Delight users.
*************************

Question: What's the most common unscalable thing founders have to do at the start?
Answer: Delighting users individually.
*************************

Question: What's one of the biggest things inexperienced founders and investors get wrong about startups?
Answer: They want to seem big too early and imitate flaws of big companies, like indifference to individual users.
*************************

